In [ ]:
# TotalSegmentator usage test

from totalsegmentator.python_api import totalsegmentator as ts

input_path = "ct.nii.gz"
output_path = "segmentations"

ts(input_path, output_path)


If you use this tool please cite: https://doi.org/10.48550/arXiv.2208.05868

Resampling...
  Resampled in 8.08s
Predicting part 1 of 5 ...
Predicting part 2 of 5 ...


In [ ]:
import SimpleITK as sitk
import os

In [12]:
# Process and format segmented files for visualization

import SimpleITK as sitk
import os

indir = "./segmentations/";
outdir = "./segmentations/mhd/";

main = sitk.ReadImage("ct.nii.gz", outputPixelType = sitk.sitkInt32)

if not os.path.exists(outdir) :
    os.makedirs(outdir)

for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file, outputPixelType = sitk.sitkInt32)

        path = outdir + file.replace(".nii.gz", ".mhd")
        mask_filter = sitk.MultiplyImageFilter()
        res = mask_filter.Execute(main, img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

In [1]:
# More lightweight solution for just the layer mask

import SimpleITK as sitk
import os

indir = "./segmentations/";
outdir = "./segmentations/mhd/";

if not os.path.exists(outdir) :
    os.makedirs(outdir)
    
for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file, outputPixelType = sitk.sitkUInt8)
        
        path = outdir + file.replace(".nii.gz", ".mhd")
        scale_filter = sitk.ShiftScaleImageFilter()
        scale_filter.SetScale(100)
        res = scale_filter.Execute(img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

In [19]:
import SimpleITK as sitk

img = sitk.ReadImage("ct.nii.gz")

sitk.WriteImage(img, "ct.mhd")

In [1]:
# Create MIP renderer

import platform
if platform.system() != 'Windows':
    import PySide2.QtSvg # Must import this before fast due to conflicting symbols
import fast # Must import FAST before rest of pyside2
from PySide2.QtWidgets import *
from PySide2.QtCore import Qt
from shiboken2 import wrapInstance
import random as rd
import functools
import keyboard
import os

# Create FAST Pipeline and window
window = fast.SimpleWindow3D.create(width=1024, height=512)\

importer = fast.ImageFileImporter.create("ct.mhd")
renderer = fast.MaximumIntensityProjection.create()
renderer.connect(importer)
window.connect(renderer)

# Run everything!
window.run()

In [2]:
# Create volume renderer

import platform
if platform.system() != 'Windows':
    import PySide2.QtSvg # Must import this before fast due to conflicting symbols
import fast # Must import FAST before rest of pyside2
from PySide2.QtWidgets import *
from PySide2.QtCore import Qt
from shiboken2 import wrapInstance
import random as rd
import functools
import keyboard
import os

# Create FAST Pipeline and window
window = fast.SimpleWindow3D.create(width=1024, height=512)\

importer = fast.ImageFileImporter.create("ct.mhd")
renderer = fast.AlphaBlendingVolumeRenderer.create()
renderer.connect(importer)
window.connect(renderer)


# Run everything!
window.run()

In [3]:
# Create visualizer

import platform
if platform.system() != 'Windows':
    import PySide2.QtSvg # Must import this before fast due to conflicting symbols
import fast # Must import FAST before rest of pyside2
from PySide2.QtWidgets import *
from PySide2.QtCore import Qt
from shiboken2 import wrapInstance
import random as rd
import functools
import keyboard
import os

segdir = "./segmentations/mhd/";

renderers = []
active = []

def import_segment(x) :
    print(x)
    
    importer = fast.ImageFileImporter.create(segdir + x)

    extraction = fast.SurfaceExtraction\
            .create(threshold=50)\
            .connect(importer)

    r = rd.uniform(0.5, 1);
    g = rd.uniform(0.5, 1);
    b = rd.uniform(0.5, 1);

    mesh = extraction.runAndGetOutputData()

    renderer = fast.TriangleRenderer\
            .create(color=fast.Color(r, g, b))\
            .connect(mesh)
    
    renderers.append(renderer)
    active.append(True)

    window.connect(renderer)
    
def toggle_segment_alpha(i):
    print(i)
    window.removeAllRenderers()
    window.connect(renderers[i])
    active[i] = True
    renderers[i].setOpacity(0, 1)
    
    changed = []
    
    for j in range(0, len(renderers)):
        if i != j and active[j]:
            window.connect(renderers[j])
            if not keyboard.is_pressed('q'):
                active[j] = False
                renderers[j].setOpacity(0, 0.05) 
                changed.append(j)
    
    for j in range(0, len(renderers)):
        if i != j and not active[j] and j not in changed:
            window.connect(renderers[j])
            renderers[j].setOpacity(0, 0.05)    
            
def reset_segment_alpha():
    for j in range(0, len(renderers)):
        renderers[j].setOpacity(0, 1)
        active[j] = True
            
    
def toggle_segment(i):
    return lambda: toggle_segment_alpha(i)

def export_mesh():
    return 0

# Create FAST Pipeline and window
window = fast.SimpleWindow3D.create(width=1024, height=512)\

# Get the underlying QtWidget of the FAST window and convert it to pyside2
mainWidget = wrapInstance(int(window.getWidget()), QWidget)

# Create GUI in Qt
layout = mainWidget.layout()
menuWidget = QWidget()
layout.addWidget(menuWidget)
menuLayout = QVBoxLayout()
menuWidget.setLayout(menuLayout)
menuLayout.setAlignment(Qt.AlignTop)
menuWidget.setFixedWidth(400)

# Create segment list
widget = QWidget()   

seglist = QVBoxLayout()
seglist.setAlignment(Qt.AlignTop)

scroll = QScrollArea()
scroll.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
scroll.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
scroll.setWidgetResizable(True)

files = os.listdir(segdir)

idx = 0

for i in range(0, len(files)) :
    if os.path.isfile(segdir + files[i]) and ".mhd" in files[i] :
        elem_button = QPushButton(files[i][:-4])
        elem_button.clicked.connect(toggle_segment(idx))
        import_segment(files[i])
        seglist.addWidget(elem_button)
        idx += 1
        
widget.setLayout(seglist)
scroll.setWidget(widget)
        
reset = QPushButton("Reset view")
reset.clicked.connect(lambda: reset_segment_alpha())
    
menuLayout.addWidget(reset)
menuLayout.addWidget(scroll)

reset = QPushButton(files[i][:-4])
        
# Run everything!
window.run()

adrenal_gland_left.mhd
adrenal_gland_right.mhd
aorta.mhd
autochthon_left.mhd
autochthon_right.mhd
clavicula_left.mhd
clavicula_right.mhd
colon.mhd
duodenum.mhd
esophagus.mhd
femur_left.mhd
femur_right.mhd
gallbladder.mhd
gluteus_maximus_left.mhd
gluteus_maximus_right.mhd
gluteus_medius_left.mhd
gluteus_medius_right.mhd
gluteus_minimus_left.mhd
gluteus_minimus_right.mhd
heart_atrium_left.mhd
heart_atrium_right.mhd
heart_myocardium.mhd
heart_ventricle_left.mhd
heart_ventricle_right.mhd
hip_left.mhd
hip_right.mhd
humerus_left.mhd
humerus_right.mhd
iliac_artery_left.mhd
iliac_artery_right.mhd
iliac_vena_left.mhd
iliac_vena_right.mhd
iliopsoas_left.mhd
iliopsoas_right.mhd
inferior_vena_cava.mhd
kidney_left.mhd
kidney_right.mhd
liver.mhd
lung_lower_lobe_left.mhd
lung_lower_lobe_right.mhd
lung_middle_lobe_right.mhd
lung_upper_lobe_left.mhd
lung_upper_lobe_right.mhd
pancreas.mhd
portal_vein_and_splenic_vein.mhd
pulmonary_artery.mhd
rib_left_1.mhd
rib_left_10.mhd
rib_left_11.mhd
rib_left_12.mhd

In [4]:
from stl import mesh as stmesh
import math
import numpy as np
import fast

indir = "./segmentations/mhd/";
outdir = "./meshes/";
# Spine
#files = [\
#    "vertebrae_L1",\
#    "vertebrae_L2",\
#    "vertebrae_L3",\
#    "vertebrae_L4",\
#    "vertebrae_L5",\
#    "vertebrae_T1",\
#    "vertebrae_T10",\
#    "vertebrae_T11",\
#    "vertebrae_T12",\
#    "vertebrae_T2",\
#    "vertebrae_T3",\
#    "vertebrae_T4",\
#    "vertebrae_T5",\
#    "vertebrae_T6",\
#    "vertebrae_T7",\
#    "vertebrae_T8",\
#    "vertebrae_T9",\
#    "sacrum"];

# Heart
files = [\
    "heart_atrium_left",\
    "heart_atrium_right",\
    "heart_myocardium",\
    "heart_ventricle_left",\
    "heart_ventricle_right"];

verts = []
faces = []

offset = 0

for file in files:
    importer = fast.ImageFileImporter.create(indir + file + ".mhd")
    smoothing = fast.GaussianSmoothing.create(stdDev=2.0).connect(importer)
    extraction = fast.SurfaceExtraction.create(threshold=50).connect(smoothing)
    mesh = extraction.runAndGetOutputData()
    access = mesh.getMeshAccess(fast.ACCESS_READ)

    for vertex in access.getVertices():
        verts.append(vertex.getPosition().reshape(1, 3))

    for face in access.getTriangles():
        faces.append([offset + face.getEndpoint1(), offset + face.getEndpoint2(), offset + face.getEndpoint3()])
        
    offset = len(verts)
    
    print(file)
    
verts = np.array(verts)
faces = np.array(faces)
    
model = stmesh.Mesh(np.zeros(faces.shape[0], dtype=stmesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        model.vectors[i][j] = verts[f[j],:]
    
model.save(outdir + "output.stl")

heart_atrium_left
heart_atrium_right
heart_myocardium
heart_ventricle_left
heart_ventricle_right


In [1]:
from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkimagereader -ifile ./ct.mhd --pipe vmtkimagewriter -ofile ./ct.vti"

res = pypes.PypeRun(args)


Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = ./ct.mhd
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = ./ct.mhd
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 0, 0]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Spacing 1.500000 1.500000 1.500000
Origin -249.032227 -85.032227 43.799999
Dimensions 319 319 441
Done executing vmtkimagereader.
Output vmtkimagereader members:
    Id = 0
    Image = vtkImageData
    RasToIjkMatrixCoefficients = [0.6666666666666666, -0.0, 0.0, 166.021484375, -0.0, 0.6666666666666666, -0.0, 56.688151

In [1]:
# Example marching cubes surface extraction without level set meshing from segmented file

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkimagereader -ifile ./segmentations/vti/aorta.vti --pipe\
        vmtkmarchingcubes -l 120.0 -ofile ./meshes/vtp/aorta.vtp --pipe\
        vmtkrenderer --pipe vmtkimageviewer --pipe vmtksurfaceviewer"

res = pypes.PypeRun(args)


Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = ./segmentations/vti/aorta.vti
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = ./segmentations/vti/aorta.vti
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 0, 0]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Reading VTK XML image file.
Spacing 1.500000 1.500000 1.500000
Origin -249.032227 -85.032227 43.799999
Dimensions 319 319 441
Done executing vmtkimagereader.
Output vmtkimagereader members:
    Id = 0
    Image = vtkImageData
    RasToIjkMatrixCoefficients = [1, 0, 0, 0, 0, 1,

In [1]:
# Segmenting the aorta via levelsets

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtklevelsetsegmentation -ifile ./segmentations/vti/aorta.vti --pipe vmtkmarchingcubes -i @.o -ofile ./meshes/vtp/aorta.vtp"

pypes.PypeRun(args)


Automatic piping vmtklevelsetsegmentation
Parsing options vmtklevelsetsegmentation
    ImageInputFileName = ./segmentations/vti/aorta.vti
Explicit piping vmtklevelsetsegmentation
Input vmtklevelsetsegmentation members:
    Id = 0
    Disabled = 0
    Image = None
    ImageInputFileName = ./segmentations/vti/aorta.vti
    FeatureImage = None
    FeatureImageInputFileName = 
    InitializationImage = None
    InitializationImageInputFileName = 
    InitialLevelSets = None
    InitialLevelSetsInputFileName = 
    LevelSets = None
    LevelSetsInputFileName = 
    LevelSetsType = geodesic
    FeatureImageType = gradient
    NegateForInitialization = 0
    SigmoidRemapping = 0
    IsoSurfaceValue = 0.0
    DerivativeSigma = 0.0
    FeatureDerivativeSigma = 0.0
    UpwindFactor = 1.0
    FWHMRadius = [1.0, 1.0, 1.0]
    FWHMBackgroundValue = 0.0
    NumberOfIterations = 0
    PropagationScaling = 0.0
    CurvatureScaling = 0.0
    AdvectionScaling = 1.0
    EdgeWeight = 0.0
    SmoothingIte

In [2]:
# Clipping mesh inlets and outlets
  
from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfaceclipper -ifile ./meshes/vtp/aorta.vtp -ofile ./meshes/vtp/aorta_clip.vtp"

res = pypes.PypeRun(args)


Automatic piping vmtksurfaceclipper
Parsing options vmtksurfaceclipper
    SurfaceInputFileName = ./meshes/vtp/aorta.vtp
    SurfaceOutputFileName = ./meshes/vtp/aorta_clip.vtp
Explicit piping vmtksurfaceclipper
Input vmtksurfaceclipper members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = ./meshes/vtp/aorta.vtp
    WidgetType = box
    Transform = None
    CleanOutput = 1
    InsideOut = 0
    Interactive = 1
    ClipArrayName = None
    ClipValue = 0.0
    vmtkRenderer = None
    SurfaceOutputFileName = ./meshes/vtp/aorta_clip.vtp
    ClippedSurfaceOutputFileName = 
    CutLinesOutputFileName = 
Reading VTK XML surface file.
Executing vmtksurfaceclipper ...
Quit renderer
Done executing vmtksurfaceclipper.
Writing VTK XML surface file.
Output vmtksurfaceclipper members:
    Id = 0
    Surface = vtkPolyData
    ClippedSurface = vtkPolyData
    CutLines = vtkPolyData
    Transform = vtkTransform


In [9]:
# Surface subdivision

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacesubdivision -ifile ./meshes/vtp/aorta_smooth.vtp -ofile ./meshes/vtp/aorta_subdiv.vtp -method butterfly"

res = pypes.PypeRun(args)


Automatic piping vmtksurfacesubdivision
Parsing options vmtksurfacesubdivision
    SurfaceInputFileName = ./meshes/vtp/aorta_smooth.vtp
    Method = butterfly
    SurfaceOutputFileName = ./meshes/vtp/aorta_subdiv.vtp
Explicit piping vmtksurfacesubdivision
Input vmtksurfacesubdivision members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = ./meshes/vtp/aorta_smooth.vtp
    NumberOfSubdivisions = 1
    Method = butterfly
    SurfaceOutputFileName = ./meshes/vtp/aorta_subdiv.vtp
Reading VTK XML surface file.
Executing vmtksurfacesubdivision ...
Done executing vmtksurfacesubdivision.
Writing VTK XML surface file.
Output vmtksurfacesubdivision members:
    Id = 0
    Surface = vtkPolyData


In [8]:
# Surface smoothing

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacesmoothing -ifile ./meshes/vtp/aorta_clip.vtp -passband 0.1 -iterations 30 -ofile ./meshes/vtp/aorta_smooth.vtp "

res = pypes.PypeRun(args)


Automatic piping vmtksurfacesmoothing
Parsing options vmtksurfacesmoothing
    SurfaceInputFileName = ./meshes/vtp/aorta_clip.vtp
    NumberOfIterations = 30
    PassBand = 0.1
    SurfaceOutputFileName = ./meshes/vtp/aorta_smooth.vtp
Explicit piping vmtksurfacesmoothing
Input vmtksurfacesmoothing members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = ./meshes/vtp/aorta_clip.vtp
    NumberOfIterations = 30
    Method = taubin
    PassBand = 0.1
    RelaxationFactor = 0.01
    BoundarySmoothing = 1
    NormalizeCoordinates = 1
    SurfaceOutputFileName = ./meshes/vtp/aorta_smooth.vtp
Reading VTK XML surface file.
Executing vmtksurfacesmoothing ...
Done executing vmtksurfacesmoothing.
Writing VTK XML surface file.
Output vmtksurfacesmoothing members:
    Id = 0
    Surface = vtkPolyData


In [7]:
# Comparison

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/aorta_clip.vtp --pipe vmtksurfacesmoothing -iterations 30 -passband 0.1 --pipe vmtkrenderer --pipe vmtksurfaceviewer -display 0 --pipe vmtksurfaceviewer -i @vmtksurfacereader.o -color 1 0 0 -display 1"

res = pypes.PypeRun(args)


Automatic piping vmtksurfacereader
Parsing options vmtksurfacereader
    InputFileName = ./meshes/vtp/aorta_clip.vtp
Explicit piping vmtksurfacereader
Input vmtksurfacereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    Surface = 0
    InputFileName = ./meshes/vtp/aorta_clip.vtp
    SurfaceOutputFileName = 
Executing vmtksurfacereader ...
Reading VTK XML surface file.
Done executing vmtksurfacereader.
Output vmtksurfacereader members:
    Id = 0
    Surface = vtkPolyData

Automatic piping vmtksurfacesmoothing
    Surface = vmtksurfacereader-0.Surface
Parsing options vmtksurfacesmoothing
    NumberOfIterations = 30
    PassBand = 0.1
Explicit piping vmtksurfacesmoothing
Input vmtksurfacesmoothing members:
    Id = 0
    Disabled = 0
    Surface = vtkPolyData
    SurfaceInputFileName = 
    NumberOfIterations = 30
    Method = taubin
    PassBand = 0.1
    RelaxationFactor = 0.01
    BoundarySmoothing = 1
    NormalizeCoordinates = 1
    SurfaceOutputFileNa

In [10]:
# Centerline computation

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkcenterlines -ifile ./meshes/vtp/aorta_smooth.vtp -ofile ./meshes/vtp/aorta_centerline.vtp"

res = pypes.PypeRun(args)


Automatic piping vmtkcenterlines
Parsing options vmtkcenterlines
    SurfaceInputFileName = ./meshes/vtp/aorta_smooth.vtp
    CenterlinesOutputFileName = ./meshes/vtp/aorta_centerline.vtp
Explicit piping vmtkcenterlines
Input vmtkcenterlines members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = ./meshes/vtp/aorta_smooth.vtp
    SeedSelectorName = pickpoint
    SourceIds = []
    TargetIds = []
    SourcePoints = []
    TargetPoints = []
    AppendEndPoints = 0
    CheckNonManifold = 0
    FlipNormals = 0
    CapDisplacement = 0.0
    DelaunayTolerance = 0.001
    RadiusArrayName = MaximumInscribedSphereRadius
    AppendEndPoints = 0
    Resampling = 0
    ResamplingStepLength = 1.0
    DelaunayTessellation = None
    SimplifyVoronoi = 0
    UseTetGen = 0
    TetGenDetectInter = 1
    CostFunction = 1/R
    vmtkRenderer = None
    PoleIds = None
    VoronoiDiagram = None
    VoronoiDiagramInputFileName = 
    StopFastMarchingOnReachingTarget = 0
    Centerl

In [1]:
# Centerline computation visualization

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/aorta_smooth.vtp --pipe vmtkcenterlines --pipe vmtkrenderer --pipe vmtksurfaceviewer -opacity 0.25 --pipe vmtksurfaceviewer -i @vmtkcenterlines.o -array MaximumInscribedSphereRadius"

res = pypes.PypeRun(args)


Automatic piping vmtksurfacereader
Parsing options vmtksurfacereader
    InputFileName = ./meshes/vtp/aorta_smooth.vtp
Explicit piping vmtksurfacereader
Input vmtksurfacereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    Surface = 0
    InputFileName = ./meshes/vtp/aorta_smooth.vtp
    SurfaceOutputFileName = 
Executing vmtksurfacereader ...
Reading VTK XML surface file.
Done executing vmtksurfacereader.
Output vmtksurfacereader members:
    Id = 0
    Surface = vtkPolyData

Automatic piping vmtkcenterlines
    Surface = vmtksurfacereader-0.Surface
Parsing options vmtkcenterlines
Explicit piping vmtkcenterlines
Input vmtkcenterlines members:
    Id = 0
    Disabled = 0
    Surface = vtkPolyData
    SurfaceInputFileName = 
    SeedSelectorName = pickpoint
    SourceIds = []
    TargetIds = []
    SourcePoints = []
    TargetPoints = []
    AppendEndPoints = 0
    CheckNonManifold = 0
    FlipNormals = 0
    CapDisplacement = 0.0
    DelaunayTolerance = 0.

In [2]:
# Centerline computation visualization

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/aorta_smooth.vtp --pipe vmtkcenterlines --pipe vmtkrenderer --pipe vmtksurfaceviewer -opacity 0.25 --pipe vmtksurfaceviewer -i @vmtkcenterlines.voronoidiagram -array MaximumInscribedSphereRadius --pipe vmtksurfaceviewer -i @vmtkcenterlines.o"

res = pypes.PypeRun(args)


Automatic piping vmtksurfacereader
Parsing options vmtksurfacereader
    InputFileName = ./meshes/vtp/aorta_smooth.vtp
Explicit piping vmtksurfacereader
Input vmtksurfacereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    Surface = 0
    InputFileName = ./meshes/vtp/aorta_smooth.vtp
    SurfaceOutputFileName = 
Executing vmtksurfacereader ...
Reading VTK XML surface file.
Done executing vmtksurfacereader.
Output vmtksurfacereader members:
    Id = 0
    Surface = vtkPolyData

Automatic piping vmtkcenterlines
    Surface = vmtksurfacereader-0.Surface
Parsing options vmtkcenterlines
Explicit piping vmtkcenterlines
Input vmtkcenterlines members:
    Id = 0
    Disabled = 0
    Surface = vtkPolyData
    SurfaceInputFileName = 
    SeedSelectorName = pickpoint
    SourceIds = []
    TargetIds = []
    SourcePoints = []
    TargetPoints = []
    AppendEndPoints = 0
    CheckNonManifold = 0
    FlipNormals = 0
    CapDisplacement = 0.0
    DelaunayTolerance = 0.

In [1]:
# Prepare centerline for Gmsh meshing

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacewriter -ifile ./meshes/vtp/aorta_centerline.vtp -ofile ./meshes/vtp/aorta_centerline.vtk"

res = pypes.PypeRun(args)


Automatic piping vmtksurfacewriter
Parsing options vmtksurfacewriter
    SurfaceInputFileName = ./meshes/vtp/aorta_centerline.vtp
    OutputFileName = ./meshes/vtp/aorta_centerline.vtk
Explicit piping vmtksurfacewriter
Input vmtksurfacewriter members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = ./meshes/vtp/aorta_centerline.vtp
    Format = 
    GuessFormat = 1
    CellData = 0
    Mode = binary
    OutputFileName = ./meshes/vtp/aorta_centerline.vtk
    OutputFileName = ./meshes/vtp/aorta_centerline.vtk
Reading VTK XML surface file.
Executing vmtksurfacewriter ...
Writing VTK surface file.
Done executing vmtksurfacewriter.
Output vmtksurfacewriter members:
    Id = 0


In [39]:
# Alternative voxel-based centerline extraction

import SimpleITK as sitk
import os
import numpy as np
import time
from scipy.ndimage import binary_erosion

file = "./segmentations/mhd/aorta.mhd"

img = sitk.ReadImage(file)

array = sitk.GetArrayFromImage(img)

eroded = array

start = time.perf_counter()

print(np.max(array))

for i in range(0, 10):
    eroded = binary_erosion(eroded)
    array += eroded
    
end = time.perf_counter()

maxval = np.max(array)

print(end - start)
print(maxval)

centerline = []

size = array.shape

for x in range(0, size[0]):
    for y in range(0, size[1]):
        for z in range(0, size[2]):
            if (array[x][y][z] == maxval):
                centerline.append((x, y, z))

100
2.7038296999999147
110


[(330, 177, 152), (331, 178, 151), (331, 178, 152), (331, 178, 153), (331, 179, 151), (331, 179, 152), (331, 179, 153), (331, 180, 151), (332, 178, 151), (332, 178, 152), (332, 178, 153), (332, 179, 151), (332, 179, 152), (332, 179, 153), (333, 179, 151), (333, 179, 152), (333, 179, 153), (334, 179, 151), (334, 179, 152), (334, 179, 153), (334, 179, 154)]
